In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/style1/128/b0/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 5000
start_epoch = 0
train_batch = 32
test_batch = 300
lr = 0.01
schedule = [1500, 3000]
momentum = 0
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b0/to_pggan/l2sp_style1' # dir
if not os.path.isdir(checkpoint):
    os.mkdir(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# sp
alpha = 0.1
beta = 0.1
fc_name = 'fc.'

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(target_dir, '100_shot_style1')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
source_model = EfficientNet.from_name(model_name, num_classes=num_classes, override_params={'dropout_rate':0.0})
model = EfficientNet.from_name(model_name, num_classes=num_classes, override_params={'dropout_rate':0.0})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    source_model.load_state_dict(torch.load(pretrained)['state_dict'])
    model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/128/b0/checkpoint.pth.tar'


In [9]:
source_model.to('cuda')
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.01M


In [10]:
for param in source_model.parameters():
    param.requires_grad = False
source_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): GroupNorm(4, 16, ep

In [11]:
source_model_weights = {}
for name, param in source_model.named_parameters():
    source_model_weights[name] = param.detach()

# Model tuning

In [12]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - source_model_weights[name]) ** 2
    return sp_loss

In [13]:
summary(model, input_size=(3,128,128),device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 129, 129]               0
Conv2dStaticSamePadding-2           [-1, 32, 64, 64]             864
         GroupNorm-3           [-1, 32, 64, 64]              64
MemoryEfficientSwish-4           [-1, 32, 64, 64]               0
         ZeroPad2d-5           [-1, 32, 66, 66]               0
Conv2dStaticSamePadding-6           [-1, 32, 64, 64]             288
         GroupNorm-7           [-1, 32, 64, 64]              64
MemoryEfficientSwish-8           [-1, 32, 64, 64]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13             [-1, 32, 1, 1]             288
         I

# Loss

In [14]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [15]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [16]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss = loss_main + alpha*loss_sp + beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss = loss_main + alpha*loss_sp + beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [18]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [19]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 50 == 0:
        test_loss, test_acc = test(val_target_loader, model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, model, criterion, epoch, use_cuda)

        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        scheduler_warmup.step()

        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)


Epoch: [1 | 5000] LR: 0.010000
1/13 | Total:0:00:01 | ETA:0:00:18 | Loss:1.8826292753219604 | top1:78.125
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:1.3888360803777522 | top1:69.03409576416016
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:1.3280127346515656 | top1:69.01042175292969


/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


107/107 | Total:0:00:15 | ETA:0:00:00 | Loss:1.2061088397124102 | top1:34.66666793823242
26/26 | Total:0:00:14 | ETA:0:00:00 | Loss:0.24459707794281152 | top1:88.82051086425781

Epoch: [2 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:12 | Loss:0.6190992593765259 | top1:75.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.6027841920202429 | top1:67.32954406738281
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.5952949300408363 | top1:66.92708587646484

Epoch: [3 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.4832412004470825 | top1:68.75
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.5321405800906095 | top1:69.03409576416016
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.5283918753266335 | top1:69.01042175292969

Epoch: [4 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.5343090295791626 | top1:78.125
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.4700188501314683 | top1:73.57954406738281
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.4652208834886551 | top1:7

1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.29398444294929504 | top1:84.375
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.2591871917247772 | top1:89.48863983154297
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.248736921697855 | top1:90.10417175292969

Epoch: [32 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:12 | Loss:0.44931262731552124 | top1:78.125
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.30106624825434253 | top1:85.79545593261719
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.3035554227729638 | top1:85.15625

Epoch: [33 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:10 | Loss:0.3571099638938904 | top1:78.125
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.2624021592465314 | top1:87.5
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.26778265461325645 | top1:87.76042175292969

Epoch: [34 | 5000] LR: 0.017000
1/13 | Total:0:00:00 | ETA:0:00:10 | Loss:0.19959987699985504 | top1:90.625
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.25575329905206506 | top1:88.06818389892578
13/13 | T

1/13 | Total:0:00:00 | ETA:0:00:10 | Loss:0.1275104135274887 | top1:93.75
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.22051634266972542 | top1:90.05682373046875
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.21312115186204514 | top1:90.36458587646484

Epoch: [61 | 5000] LR: 0.024000
1/13 | Total:0:00:00 | ETA:0:00:10 | Loss:0.07671092450618744 | top1:96.875
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.18684896352616223 | top1:93.46591186523438
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.18206624376277128 | top1:93.48958587646484

Epoch: [62 | 5000] LR: 0.024000
1/13 | Total:0:00:00 | ETA:0:00:10 | Loss:0.08205749839544296 | top1:96.875
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.13209335099567066 | top1:94.6022720336914
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.12737622981270155 | top1:94.79167175292969

Epoch: [63 | 5000] LR: 0.024000
1/13 | Total:0:00:01 | ETA:0:00:15 | Loss:0.18337690830230713 | top1:87.5
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.20145569199865515 | top1:90.6

...

11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.00678881359371272 | top1:99.43182373046875
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.006232504112025102 | top1:99.47917175292969

Epoch: [4863 | 5000] LR: 0.000000
1/13 | Total:0:00:01 | ETA:0:00:13 | Loss:0.010870678350329399 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.005236014893109148 | top1:99.71591186523438
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.004876664994905393 | top1:99.73958587646484

Epoch: [4864 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:09 | Loss:0.0006503686308860779 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.006116296249357137 | top1:99.71591186523438
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.005645753350108862 | top1:99.73958587646484

Epoch: [4865 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:10 | Loss:0.04435104876756668 | top1:96.875
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.011026927185329523 | top1:99.43182373046875
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0

11/13 | Total:0:00:01 | ETA:0:00:01 | Loss:0.009051069447940046 | top1:99.71591186523438
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.008515677570054928 | top1:99.73958587646484

Epoch: [4892 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:09 | Loss:0.002686087042093277 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.003722490573471243 | top1:99.71591186523438
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0034411043549577394 | top1:99.73958587646484

Epoch: [4893 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:07 | Loss:0.00024837255477905273 | top1:100.0
11/13 | Total:0:00:01 | ETA:0:00:01 | Loss:0.0015380330045114863 | top1:100.0
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0014306885811189811 | top1:100.0

Epoch: [4894 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:07 | Loss:0.0008319392800331116 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.007757224989208308 | top1:99.43182373046875
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0077543705701828 

1/13 | Total:0:00:01 | ETA:0:00:13 | Loss:0.049219537526369095 | top1:96.875
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.010997404429045591 | top1:99.43182373046875
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.010122541959087053 | top1:99.47917175292969

Epoch: [4921 | 5000] LR: 0.000000
1/13 | Total:0:00:01 | ETA:0:00:13 | Loss:0.0002480745315551758 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.006444167853756385 | top1:99.71591186523438
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.01626478771989544 | top1:99.47917175292969

Epoch: [4922 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.0005931556224822998 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.004034582864154469 | top1:99.71591186523438
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.003880268894135952 | top1:99.73958587646484

Epoch: [4923 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:12 | Loss:0.0011279545724391937 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.002158762

11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.024042955176396805 | top1:99.43182373046875
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.0232004940820237 | top1:99.47917175292969

Epoch: [4950 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:10 | Loss:0.00041894614696502686 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0017789475281130183 | top1:100.0
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.0017045259786148865 | top1:100.0

Epoch: [4951 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:11 | Loss:0.0005604252219200134 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.02228020995178006 | top1:99.1477279663086
13/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.02052456559613347 | top1:99.21875
107/107 | Total:0:00:18 | ETA:0:00:00 | Loss:0.5439881752027529 | top1:86.61370849609375
26/26 | Total:0:00:15 | ETA:0:00:00 | Loss:0.41378009433929736 | top1:89.025634765625

Epoch: [4952 | 5000] LR: 0.000799
1/13 | Total:0:00:00 | ETA:0:00:12 | Loss:0.00048705190420150757 | 

1/13 | Total:0:00:01 | ETA:0:00:13 | Loss:0.0006159916520118713 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.021781381050294094 | top1:99.1477279663086
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.02107781342541178 | top1:99.21875

Epoch: [4979 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:12 | Loss:0.0016368478536605835 | top1:100.0
11/13 | Total:0:00:02 | ETA:0:00:01 | Loss:0.008857814764434641 | top1:99.71591186523438
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.008143615288039049 | top1:99.73958587646484

Epoch: [4980 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:12 | Loss:0.000672951340675354 | top1:100.0
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.0011481419205665588 | top1:100.0
13/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.0010716971009969711 | top1:100.0

Epoch: [4981 | 5000] LR: 0.000000
1/13 | Total:0:00:00 | ETA:0:00:12 | Loss:0.05026781186461449 | top1:96.875
11/13 | Total:0:00:03 | ETA:0:00:01 | Loss:0.01866161213679747 | top1:99.43182373046875
